<a href="https://colab.research.google.com/github/dfvnunes/GTM-Setup/blob/main/Assist%C3%AAncias_Liga_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import re

In [2]:
def abreviar_clubes_b_duas_colunas(df_drop, coluna1='Jogo', coluna2='Liga 2'):
    """
    Substitui os nomes completos dos clubes B pelos seus nomes abreviados
    nas duas colunas especificadas do DataFrame.

    Args:
        df (pd.DataFrame): O DataFrame do pandas.
        coluna1 (str): O nome da primeira coluna onde os clubes B estão presentes.
        coluna2 (str): O nome da segunda coluna onde os clubes B estão presentes.

    Returns:
        pd.DataFrame: O DataFrame com os nomes abreviados dos clubes B.
    """

    clubes_b = {
        'SL Benfica B': 'SLB B',
        'Sporting CP B': 'SCP B',
        'FC Porto B': 'FCP B',
        'SC Braga B': 'SCB B',
        'Vitória SC B': 'VSC B',
        'Marítimo B': 'CSM B'
    }

    # Expressão regular para extrair os nomes dos clubes B
    regex = r'(SL Benfica B|Sporting CP B|FC Porto B|SC Braga B|Vitória SC B|Marítimo B)'

    # Extrai e substitui os nomes dos clubes B na coluna 1
    df_drop[coluna1] = df_drop[coluna1].apply(lambda x: re.sub(regex, lambda m: clubes_b[m.group()], str(x)))

    # Substitui os nomes completos pelos abreviados na coluna 2
    df_drop[coluna2] = df_drop[coluna2].replace(clubes_b)

    return df_drop

In [3]:
def comparar_substrings(df_drop, Jogo, Liga1, Liga):

    def verificar_substring(row):
        # Remove NaN values from the Liga1 column before comparison
        liga1_values = [value for value in df_drop[Liga1].unique() if pd.notna(value)]

        # Check if any non-NaN value from 'Liga1' column is present in 'Jogo' column
        if any(liga1_value in row[Jogo] for liga1_value in liga1_values):
            return "Liga 1"
        else:
            return "Liga 2"

    df_drop[Liga] = df_drop.apply(verificar_substring, axis=1)
    return df_drop

In [4]:
def extrair_numero_jornada(df_drop, coluna='Jornada'):
    """
    Extrai o número da jornada da coluna especificada.

    Args:
        df (pd.DataFrame): O DataFrame do pandas.
        coluna (str): O nome da coluna com os valores da jornada.

    Returns:
        pd.DataFrame: O DataFrame com os números das jornadas extraídos.
    """

    def extrair_numero(valor):
        """Extrai o número da jornada de um valor."""
        match = re.search(r'(\d+)', str(valor))
        if match:
            return match.group(1)
        return valor  # Retorna o valor original se não encontrar um número

    df_drop[coluna] = df_drop[coluna].apply(extrair_numero)
    return df_drop

In [13]:
excel_file = pd.ExcelFile('/content/Por_Jogo_upload.xlsx')

In [14]:
with pd.ExcelWriter('Assistencias por Jogo.xlsx', mode='w') as writer:  # Substitua 'resultados.xlsx' pelo nome desejado
    # Loop para iterar pelas folhas

     for sheet_name in excel_file.sheet_names:
         # Lê a folha atual
         df = excel_file.parse(sheet_name)

         # Aplica as funções ao DataFrame da folha
         df_drop = df.drop('Ranking', axis=1)
         df_drop = abreviar_clubes_b_duas_colunas(df_drop, coluna1='Jogo', coluna2='Liga 2')
         df_drop = df_drop.rename(columns={'Liga 1': 'Liga1'})
         df_drop = df_drop.rename(columns={'Liga 2': 'Liga2'})
         df_drop['Jogo'] = df_drop['Jogo'].astype(str)
         df_drop['Liga1'] = df_drop['Liga1'].astype(str)
         df_liga = comparar_substrings(df_drop, 'Jogo', 'Liga1', 'Liga')
         df_drop = df_drop.drop(['Liga1','Liga2'], axis=1)
         df_drop = df_drop[~df_drop['Jornada'].isin(['Liga 1', 'Liga 2'])]
         df_drop = extrair_numero_jornada(df_drop, coluna='Jornada')


         # Guarda o resultado na folha correspondente
         df_drop.to_excel(writer, sheet_name=sheet_name, index=False, header=True)
